In [43]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [53]:
# root = "/kaggle/input/jpx-tokyo-stock-exchange-prediction/"

# stock_prices = pd.read_csv(os.path.join(dirname, "stock_prices.csv"))

train = pd.read_csv("train.csv")



In [52]:
train[["matched_size", "imbalance_size", "imbalance_buy_sell_flag"]]

train[train.imbalance_size == 0]

# train["test_ratio"] = train.imbalance_buy_sell_flag*train.matched_size/train.imbalance_size

,stock_id,date_id,seconds_in_bucket,imbalance_size,imbalance_buy_sell_flag,reference_price,matched_size,far_price,near_price,bid_price,bid_size,ask_price,ask_size,wap,target,time_id,row_id
5,5,0,0,0.0,0,1.000635,1.355288e+07,NaN,NaN,0.999779,1962.72,1.000635,5647.65,1.000000,6.779432,0,0_0_5
19,19,0,0,0.0,0,0.999387,2.815451e+06,NaN,NaN,0.999218,17775.00,1.001298,29567.92,1.000000,4.299879,0,0_0_19
35,35,0,0,0.0,0,0.999841,6.301295e+06,NaN,NaN,0.999676,59482.80,1.000007,1209.40,1.000000,0.189543,0,0_0_35
37,37,0,0,0.0,0,1.000149,1.108548e+07,NaN,NaN,0.999910,19682.66,1.000340,74365.40,1.000000,-0.429750,0,0_0_37
39,39,0,0,0.0,0,0.999539,5.760194e+06,NaN,NaN,0.999539,57147.72,1.000078,9724.25,1.000000,-4.699826,0,0_0_39
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5237938,158,480,540,0.0,0,0.999648,8.578806e+06,0.999648,0.999648,0.999648,47588.31,0.999874,38768.09,0.999773,-2.260208,26454,480_540_158
5237948,168,480,540,0.0,0,0.998735,4.088902e+08,0.998735,0.998735,0.998735,115812.96,0.998811,741364.12,0.998745,-1.919866,26454,480_540_168
5237949,169,480,540,0.0,0,1.000357,1.345633e+07,1.000357,1.000357,1.000357,12959.60,1.000697,58986.20,1.000418,-1.519918,26454,480_540_169
5237954,174,480,540,0.0,0,1.000762,4.826936e+06,1.000762,1.000762,1.000762,297950.40,1.001412,170508.80,1.001176,16.599894,26454,480_540_174


In [54]:

def make_data(df, train=0):
	data = df.set_index(["stock_id", "date_id", "seconds_in_bucket"])

	data["imbalance_buy_sell"] = data.imbalance_size * data.imbalance_buy_sell_flag

	params = ["imbalance_buy_sell", "reference_price", "matched_size", "wap_logreturns"]

	data["wap_logreturns"] = np.log(data.wap/data.wap.shift(1))

	data["matched_ratio"] = data.imbalance_buy_sell_flag*data.imbalance_size/data.matched_size

	data = data.dropna(subset=params)

	X = np.array([data.wap_logreturns, data.matched_ratio]).reshape(-1,2)

	if train == 0:
		y = np.array(data.target).reshape(-1,1)
		return X, y

	return X

# def make_data(df, train=0):
# 	data = df.set_index(["stock_id", "date_id", "seconds_in_bucket"])

# 	data["imbalance_buy_sell"] = data.imbalance_size * data.imbalance_buy_sell_flag

# 	params = ["imbalance_buy_sell", "reference_price", "matched_size"]

# 	data = data.dropna(subset=params)
# 	X = np.array([data.wap, data.imbalance_buy_sell]).reshape(-1,2)

# 	if train == 0:
# 		y = np.array(data.target).reshape(-1,1)
# 		return X, y

# 	return X


In [55]:


X, y = make_data(train)

X_train, X_test, y_train, y_test = train_test_split(X,y ,
                                   random_state=104,
                                   test_size=0.25,
                                   shuffle=True)

In [56]:
model = LinearRegression()
model.fit(X_train,y_train)


LinearRegression()

In [57]:
model.coef_

predict_target = model.predict(X)

In [58]:
# train[train.seconds_in_bucket > 10].head()
# train.loc[train.stock_id==1,['target', 'time_id', 'stock_id', 'seconds_in_bucket']]
train[train.stock_id==1]['seconds_in_bucket'].unique()

array([  0,  10,  20,  30,  40,  50,  60,  70,  80,  90, 100, 110, 120,
       130, 140, 150, 160, 170, 180, 190, 200, 210, 220, 230, 240, 250,
       260, 270, 280, 290, 300, 310, 320, 330, 340, 350, 360, 370, 380,
       390, 400, 410, 420, 430, 440, 450, 460, 470, 480, 490, 500, 510,
       520, 530, 540])

In [59]:
pd.DataFrame(predict_target).describe()

,0
count,5.237539e+06
mean,-4.830533e-02
std,5.443517e-03
min,-5.038790e-01
25%,-4.839675e-02
50%,-4.833692e-02
75%,-4.827578e-02
max,6.678699e-01


In [60]:

stats.describe(predict_target)



DescribeResult(nobs=5237539, minmax=(array([-0.50387904]), array([0.66786987])), mean=array([-0.04830533]), variance=array([2.96318773e-05]), skewness=array([2.34031094]), kurtosis=array([518.43800575]))

In [61]:
import optiver2023

env = optiver2023.make_env()
iter_test = env.iter_test()


ModuleNotFoundError: No module named 'optiver2023.competition'

In [ ]:
counter = 0
for (test, revealed_targets, sample_prediction) in iter_test:
    if counter == 0:
        print(test.head(3))
        print(revealed_targets.head(3))
        print(sample_prediction.head(3))

    X = make_data(test, train=1)
    sample_prediction['target'] = model.predict(X)
    sample_prediction['target'] = 0
    env.predict(sample_prediction)
    counter += 1

This version of the API is not optimized and should not be used to estimate the runtime of your code on the hidden test set.
   stock_id  date_id  seconds_in_bucket  imbalance_size  \
0         0      478                  0      3753451.43   
1         1      478                  0       985977.11   
2         2      478                  0       599128.74   

   imbalance_buy_sell_flag  reference_price  matched_size  far_price  \
0                       -1         0.999875   11548975.43        NaN   
1                       -1         1.000245    3850033.97        NaN   
2                        1         1.000584    4359198.25        NaN   

   near_price  bid_price  bid_size  ask_price  ask_size  wap   row_id  
0         NaN   0.999875  22940.00   1.000050   9177.60  1.0  478_0_0  
1         NaN   0.999940   1967.90   1.000601  19692.00  1.0  478_0_1  
2         NaN   0.999918   4488.22   1.000636  34955.12  1.0  478_0_2  
   stock_id  date_id  seconds_in_bucket  revealed_target  rev

In [ ]:
# for (prices, options, financials, trades, secondary_prices, sample_prediction) in iter_test:
#     prices.head()
#     x_test = featuring_2(prices)
#     y_pred = model.predict(x_test)
#     sample_prediction['Target'] = y_pred
#     sample_prediction = sample_prediction.sort_values(by = "Target", ascending = False)
#     sample_prediction['Rank'] = np.arange(len(sample_prediction.index))
#     sample_prediction = sample_prediction.sort_values(by = "SecuritiesCode", ascending = True)
#     sample_prediction.drop(["Target"], axis = 1)
#     submission = sample_prediction[["Date", "SecuritiesCode", "Rank"]]
#     env.predict(submission)

In [39]:
print(sample_prediction)

counter

NameError: name 'sample_prediction' is not defined

In [38]:
!head submission.csv
sub = pd.read_csv("submission.csv")


head: submission.csv: No such file or directory


FileNotFoundError: [Errno 2] No such file or directory: 'submission.csv'

In [29]:
sub[sub.target == 'target']

# type(sub.targetpd.to_numeric(df['id'], errors='coerce').notnull())
sub.target = pd.to_numeric(sub.target, errors='coerce')
sub = sub[sub.target.notnull()]


In [31]:
sub.target.max()

0.0971120314874603

In [1]:
!ls

example_test_files    optiver2023			 train.csv
__init__.py	      public_timeseries_testing_util.py
kaggle_optiver.ipynb  submission.csv
